# Access large areas

In the previous notebook `0_access_worldcover_products`, ESA WorldCover 2020 data was loaded and analysed over a small area of interest.

In this notebook, WorldCover data in the spatial extend of an entire country will be searched, downloaded and analysed.

The entire country of Belgium will be analysed. As the country area interestects with multiple WorldCover rasters, the `rio-tiler` package will be used and is therefore required to run this notebook.

## Find the STAC products for the area of interest

In [4]:
# Load the area of interest geometry (belgium)
import geopandas as gpd

# Load countries border dataset
countries_dataset = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

belgium_entry = countries_dataset.loc[countries_dataset['name'] == 'Belgium']
belgium_entry

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
129,11491346,Europe,Belgium,BEL,508600.0,"POLYGON ((6.15666 50.80372, 6.04307 50.12805, ..."


In [5]:
bbox = list(belgium_entry.geometry.bounds.values.squeeze())
bbox

[2.5135730322461427, 49.529483547557504, 6.15665815595878, 51.47502370869813]

In [6]:
from rio_tiler.io import STACReader
from rio_tiler.profiles import img_profiles
from rio_tiler.models import ImageData
import pystac_client
import geopandas as gpd

# Log in to the terrascope STAC API
TERRASCOPE_STAC_API = 'https://services.terrascope.be/stac/'
COLLECTION_ID = 'urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1'

client = pystac_client.Client.open(TERRASCOPE_STAC_API)
search = client.search(
    max_items=100,
    collections=[COLLECTION_ID],
    bbox = bbox
)

result_items = search.get_all_items().items

result_items

[<Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E000>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E003>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E006>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E000>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E003>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E006>]

In [7]:
# We got 3 result items, let's get the access links to the rasters
result_items_assets = list(map(lambda item: item.assets.get('ESA_WORLDCOVER_10M_MAP').href, result_items))
result_items_assets

['s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E000_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E003_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E006_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E000_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E003_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E006_Map.tif']

## Parallel access of tiles trough rasterio

Using `rasterio`, we will load all the tiles in multiple threads

In [ ]:
import rasterio
import os
import threading

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

# Initialize result array
results = [None] * len(result_items_assets)

# Function to load the dataset from a remote link
def load_dataset(index, path):
    with rasterio.open(path, 'r') as src:
        # Window on our interest area, we don't want to download all the entire tiles.
        interest_window = rasterio.windows.from_bounds(*bbox, transform=src.transform)

        data = src.read(1, 
                        out_dtype=rasterio.uint8,
                        window=interest_window)
        results[index] = {'data': data, 
                          'crs': src.crs,
                          'transform': src.transform,
                          'bounds': src.bounds}

# Initialzes all the threads with the asset link parameters 
thread_pool = [threading.Thread(target=load_dataset, args=(index, asset,)) 
               for index, asset in enumerate(result_items_assets)]

# Starts all the threads
for thread in thread_pool:
    thread.start()
    
# Wait that all the threads have finished their job
for thread in thread_pool:
    thread.join()
    
results

# Recreate the full image from the extracted data